In [ ]:
!python -m pip install shioaji

     |████████████████████████████████| 5.9 MB 4.3 MB/s 
     |████████████████████████████████| 93 kB 2.2 MB/s 
     |████████████████████████████████| 18.1 MB 218 kB/s 
     |████████████████████████████████| 10.9 MB 44.4 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 243 kB 48.6 MB/s 
     |████████████████████████████████| 208 kB 57.6 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.1
    Uninstalling importlib-metadata-4.11.1:
      Successfully uninstalled importlib-metadata-4.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.

In [ ]:
from platform import python_version
print(python_version())


3.7.12


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shioaji as sj

api = sj.Shioaji()
accounts = api.login("J120420374", "celia5818")



Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [ ]:
api.activate_ca(
    ca_path="/content/drive/MyDrive/SinoPac/Sinopac.pfx",
    ca_passwd="J120420374",
    person_id="J120420374",
)


True

accounts = api.list_accounts()
print(accounts)


In [ ]:
# Stock default account
print(api.stock_account)

# Futures default account
print(api.futopt_account)


person_id='J120420374' broker_id='9A97' account_id='0296637' signed=True username='黃瑞柳\u3000\u3000'
person_id='J120420374' broker_id='F002000' account_id='1641626' signed=True username='黃瑞柳'


##Subscribe

In [ ]:
contract_fut_2317_03 = api.Contracts.Futures.DHF['DHF202203'] #鴻海期貨202203 

'''
api.quote.subscribe(
    contract=contract_fut_2317_03, 
    quote_type="tick", # {'tick', 'bidask'}
    version = sj.constant.QuoteVersion.v0, # or 'v1'
)
'''

api.quote.subscribe(
    contract=contract_fut_2317_03, 
    quote_type = "bidask",
    version = sj.constant.QuoteVersion.v0 # or 'v1'
)


print(contract_fut_2317_03)

code='DHFC2' symbol='DHF202203' name='鴻海期貨03' category='DHF' delivery_month='202203' delivery_date='2022/03/16' underlying_kind='S' underlying_code='2317' unit=1 limit_up=114.5 limit_down=94.1 reference=104.5 update_date='2022/02/23'
Response Code: 200 | Event Code: 16 | Info: Subscription Already Exists - Topic 'Q/*/DHFC2' Subscribe: Response code: '400'; Flags: 'c';  | Event: Subscribe or Unsubscribe ok


##Quote callback

In [ ]:
from shioaji import TickFOPv1, BidAskFOPv1, Exchange

@api.on_tick_fop_v1()
def quote_callback(exchange:Exchange, tick:TickFOPv1):
    print(f"Exchange: {exchange}, Tick: {tick}")
   
@api.on_bidask_fop_v1()
def quote_callback(exchange:Exchange, bidask:BidAskFOPv1):
    print(f"Exchange: {exchange}, BidAsk: {bidask}")

## --- Place Order


In [ ]:
# Set order callback
def place_cb(stat, msg):
    print('\n [__my_place_callback__]')
    print(stat, msg)
api.set_order_callback(place_cb)

# futures order
order_fut_1 = api.Order(
    action=sj.constant.Action.Buy,
    price=99.5,
    quantity=2,
    price_type=sj.constant.StockPriceType.LMT,
    order_type=sj.constant.FuturesOrderType.ROD, 
    octype=sj.constant.FuturesOCType.Auto,
    account=api.futopt_account
)


In [ ]:
# Place Order
trade = api.place_order(contract_fut_2317_02, order_fut_1)

## --- Cancel Order



In [ ]:
api.update_status(api.futopt_account)
api.cancel_order(trade)


Trade(contract=Future(code='DHFB2', symbol='DHF202202', name='鴻海期貨02', category='DHF', delivery_month='202202', delivery_date='2022/02/16', underlying_kind='S', underlying_code='2317', unit=1, limit_up=113.0, limit_down=92.7, reference=103.0, update_date='2022/01/26'), order=Order(action=<Action.Buy: 'Buy'>, price=99.0, quantity=2, id='45973c3e', seqno='118141', ordno='tn00V', account=Account(account_type=<AccountType.Future: 'F'>, person_id='J120420374', broker_id='F002000', account_id='1641626', signed=True), price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>, octype=<FuturesOCType.New: 'New'>), status=OrderStatus(id='45973c3e', status=<Status.Cancelled: 'Cancelled'>, status_code='8014', order_datetime=datetime.datetime(2022, 2, 7, 13, 41, 51), msg='預約完成', modified_price=99.0, cancel_quantity=2, deals=[]))

__my_place_callback__
OrderState.FOrder {'operation': {'op_type': 'Cancel', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '45973c3e', 'seqno': '118141', 'ordno': 'tn00V', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002000', 'account_id': '1641626', 'signed': True}, 'action': 'Buy', 'price': 99.0, 'quantity': 0, 'order_type': 'ROD', 'market_type': 'Day', 'oc_type': 'New', 'subaccount': ''}, 'status': {'id': '45973c3e', 'exchange_ts': 1644241583, 'modified_price': 0.0, 'cancel_quantity': 0, 'order_quantity': 0}, 'contract': {'security_type': 'FUT', 'code': 'DHF', 'exchange': 'TIM', 'delivery_month': '202202', 'delivery_date': '', 'strike_price': 0.0, 'option_right': 'Future'}}


## ---Update Price

In [ ]:
api.update_status(api.futopt_account)
api.update_order(trade=trade, price=100.5)

Trade(contract=Future(code='DHFB2', symbol='DHF202202', name='鴻海期貨02', category='DHF', delivery_month='202202', delivery_date='2022/02/16', underlying_kind='S', underlying_code='2317', unit=1, limit_up=113.0, limit_down=92.7, reference=103.0, update_date='2022/01/26'), order=Order(action=<Action.Buy: 'Buy'>, price=100.0, quantity=1, id='0aaa0058', seqno='118134', ordno='tn00U', account=Account(account_type=<AccountType.Future: 'F'>, person_id='J120420374', broker_id='F002000', account_id='1641626', signed=True), price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>, octype=<FuturesOCType.New: 'New'>), status=OrderStatus(id='0aaa0058', status=<Status.PreSubmitted: 'PreSubmitted'>, status_code='8014', order_datetime=datetime.datetime(2022, 2, 7, 13, 37, 8), msg='預約完成', modified_price=101.0, deals=[]))

__my_place_callback__
OrderState.FOrder {'operation': {'op_type': 'UpdatePrice', 'op_code': '00', 'op_msg': ''}, 'order': {'id': '0aaa0058', 'seqno': '118134', 'ordno': 'tn00U', 'account': {'account_type': 'F', 'person_id': '', 'broker_id': 'F002000', 'account_id': '1641626', 'signed': True}, 'action': 'Buy', 'price': 100.5, 'quantity': 1, 'order_type': 'ROD', 'market_type': 'Day', 'oc_type': 'New', 'subaccount': ''}, 'status': {'id': '0aaa0058', 'exchange_ts': 1644241135, 'modified_price': 100.5, 'cancel_quantity': 0, 'order_quantity': 1}, 'contract': {'security_type': 'FUT', 'code': 'DHF', 'exchange': 'TIM', 'delivery_month': '202202', 'delivery_date': '', 'strike_price': 0.0, 'option_right': 'Future'}}


## ---Update Quantity

In [ ]:
api.update_status(api.futopt_account)
api.update_order(trade=trade, qty=0)

Trade(contract=Future(code='DHFB2', symbol='DHF202202', name='鴻海期貨02', category='DHF', delivery_month='202202', delivery_date='2022/02/16', underlying_kind='S', underlying_code='2317', unit=1, limit_up=113.0, limit_down=92.7, reference=103.0, update_date='2022/01/26'), order=Order(action=<Action.Buy: 'Buy'>, price=99.5, quantity=2, id='703e00f4', seqno='118147', ordno='tn00W', account=Account(account_type=<AccountType.Future: 'F'>, person_id='J120420374', broker_id='F002000', account_id='1641626', signed=True), price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>, octype=<FuturesOCType.New: 'New'>), status=OrderStatus(id='703e00f4', status=<Status.PreSubmitted: 'PreSubmitted'>, status_code='8014', order_datetime=datetime.datetime(2022, 2, 7, 13, 47, 14), msg='預約完成', modified_price=99.5, cancel_quantity=1, deals=[]))

## --- Deal Event: Filled & Filling
Deal event occurs when the order is full/partially filled. Note that you "may" recieve the deal event sooner than the order event due to message priority in exchange.